<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/06_fine_tuning_llm/0_quantization_memory_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install necessary libraries (quiet mode)
!pip install -q transformers torch accelerate bitsandbytes

In [ ]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import os

In [ ]:
def get_model_size_mb(model):
    """
    Calculate the memory footprint of a model in Megabytes (MB).
    Handles both Hugging Face models and raw PyTorch quantized modules.
    """
    # If it's a Hugging Face model, use their built-in efficient method
    if hasattr(model, "get_memory_footprint"):
        return model.get_memory_footprint() / 1024 / 1024

    # Fallback for raw PyTorch quantized models (manual calculation)
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])
    return (mem_params + mem_buffers) / 1024 / 1024

In [ ]:
def flush():
    """Cleans up memory to ensure a fair comparison."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
# Define a small model for the demo (OPT-125M is small and fast to download)
model_id = "facebook/opt-125m"
print(f"--- Comparision for Model: {model_id} ---\n")

In [ ]:
# ==========================================
# PHASE 1: Load Non-Quantized Model (FP32)
# ==========================================
print("1. Loading Standard Model (Float32)...")
try:
    model_std = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32)
    size_std = get_model_size_mb(model_std)
    print(f"   Standard Model Size: {size_std:.2f} MB")

    # Cleanup to free memory for next step
    del model_std
    flush()
except Exception as e:
    print(f"Error loading standard model: {e}")

print("-" * 30)

In [ ]:
# ==========================================
# PHASE 2: Load Quantized Model
# ==========================================
if torch.cuda.is_available():
    # GPU PATH: Use bitsandbytes 4-bit Quantization (Modern LLM approach)
    print("2. GPU Detected. Loading with 4-bit Quantization (bitsandbytes)...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    try:
        model_quant = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )
        size_quant = get_model_size_mb(model_quant)
        print(f"   Quantized Model Size (4-bit): {size_quant:.2f} MB")

        # Calculate savings
        savings = size_std / size_quant
        print(f"\nResult: The quantized model is ~{savings:.1f}x smaller than the standard model.")

    except Exception as e:
        print(f"Error during GPU quantization: {e}")

else:
    # CPU PATH: Use PyTorch Dynamic Quantization (Int8)
    print("2. No GPU Detected. Using PyTorch Dynamic Quantization (Int8)...")

    try:
        # Reload standard model to apply dynamic quantization
        model_to_quantize = AutoModelForCausalLM.from_pretrained(model_id)

        # Apply dynamic quantization to Linear layers
        model_quant = torch.quantization.quantize_dynamic(
            model_to_quantize,
            {torch.nn.Linear},
            dtype=torch.qint8
        )

        size_quant = get_model_size_mb(model_quant)
        print(f"   Quantized Model Size (Int8): {size_quant:.2f} MB")

        # Calculate savings
        savings = size_std / size_quant
        print(f"\nResult: The quantized model is ~{savings:.1f}x smaller than the standard model.")

    except Exception as e:
        print(f"Error during CPU quantization: {e}")